In [2]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(4):
    # Directory containing your event files
    log_dir = f"/home/<User>/repos/csg_is/calculating_scores/CFL/4Nodes_non_iid/metrics/participant_{i}"
    date_str = "15/07/2024 16:00:21" # minus two hours for timezone

    date_obj = datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

   step  value                     timestamp                    tag
0     0    0.0 2024-07-17 14:00:33.001470720                  Round
1    22    0.0 2024-07-17 14:00:43.140211712  Resources/CPU_percent
2    32    0.5 2024-07-17 14:00:53.223925248  Resources/CPU_percent
3    42    0.6 2024-07-17 14:01:03.277425920  Resources/CPU_percent
4    52    0.6 2024-07-17 14:01:13.397073920  Resources/CPU_percent
2024-07-17 14:16:12 2024-07-17 14:00:33
                step     value                     timestamp
tag                                                         
Test/Accuracy   5372  0.626268 2024-07-17 14:16:05.223431424
Test/F1Score    5372  0.574700 2024-07-17 14:16:05.223461376
Test/Loss       5372  2.306607 2024-07-17 14:16:05.223400192
Test/Precision  5372  0.594847 2024-07-17 14:16:05.223441920
Test/Recall     5372  0.626268 2024-07-17 14:16:05.223451136
   step     value                     timestamp         tag
0  3722  0.218036 2024-07-17 14:13:21.538951424  Train/Loss
1  37

In [3]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,5372,0.626268,2024-07-17 14:16:05.223431424,participant_0
Test/F1Score,5372,0.574700,2024-07-17 14:16:05.223461376,participant_0
Test/Loss,5372,2.306607,2024-07-17 14:16:05.223400192,participant_0
Test/Precision,5372,0.594847,2024-07-17 14:16:05.223441920,participant_0
Test/Recall,5372,0.626268,2024-07-17 14:16:05.223451136,participant_0
Test/Accuracy,5481,0.904302,2024-07-17 14:16:16.375391232,participant_1
Test/F1Score,5481,0.891048,2024-07-17 14:16:16.375421696,participant_1
Test/Loss,5481,0.191883,2024-07-17 14:16:16.375360000,participant_1
Test/Precision,5481,0.905324,2024-07-17 14:16:16.375402240,participant_1


In [4]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_26647/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,5437.0,0.7656
Test/F1Score,5437.0,0.7331
Test/Loss,5437.0,1.2525
Test/Precision,5437.0,0.7498
Test/Recall,5437.0,0.7656


In [6]:
concat_df.groupby("tag").std().round(2)

/tmp/ipykernel_26647/582865284.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").std().round(2)


,step,value
tag,,
Test/Accuracy,54.04,0.16
Test/F1Score,54.04,0.18
Test/Loss,54.04,1.22
Test/Precision,54.04,0.18
Test/Recall,54.04,0.16


In [ ]:
test_results = concat_df.groupby("tag").mean().round(4)
test_results[test_results.index.str.startswith("Test/")]

In [4]:
concat_df.to_csv("results_csvs/dfl_15_ring.csv")